## Imports

In [ ]:
# utils
import os
import sys
from pathlib import Path
PROJECT_DIR = Path.cwd().parent
sys.path.append(str(PROJECT_DIR))

# basics
import numpy as np
import pandas as pd
from tqdm import tqdm

# metrics
from sklearn.metrics import r2_score

# viz
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import seaborn as sns

# metrics
from utils import config
from utils.reader import read_file_yaml

## Parameters

In [ ]:
path_results = (
    PROJECT_DIR
    / "results"
)
path_outputs = (
    PROJECT_DIR
    / "outputs"
)
file_path_parameters = (
    PROJECT_DIR
    / "conf" 
    / "parameters.yml"
)

file_path_abi_diff = {
    i: {
        name: {
          _param: path_results / i / name / "params" / Path(_param+".csv") for _param in ["abilities","diff_disc"]
        } for name in os.listdir(path_results / i)
    } for i in os.listdir(path_results)
}

file_path_pij = {
    i: {
        name: {
          _param: path_results / i / name / "pij" / Path(_param+".csv") for _param in ["pij_true","pij_pred"]
        } for name in os.listdir(path_results / i)
    } for i in os.listdir(path_results)
}

n_random = np.sort([int(i.replace("random_n",""))for i in os.listdir(path_results) if ".placehold"not in i])
path_random = [
    "random_n"+str(i) for i in n_random
]

params = read_file_yaml(file_path_parameters)

## Read datasets

In [ ]:
## params
data_params = {
    i_random: {
      j_name: {
          k_param: pd.read_csv(k_content, index_col = 0) for k_param, k_content in j_content.items()
      } for j_name, j_content in i_content.items()  
    } for i_random, i_content in file_path_abi_diff.items()
}

In [ ]:
## pij
data_pij = {
    i_random: {
      j_name: {
          k_param: pd.read_csv(k_content, index_col = 0) for k_param, k_content in j_content.items()
      } for j_name, j_content in i_content.items()  
    } for i_random, i_content in file_path_pij.items()
}

## Plot

In [ ]:
pij_true = data_pij["random_n1"]["aniso"]["pij_true"].copy()
abilities = data_params["random_n1"]["aniso"]["abilities"].copy()
diff_disc = data_params["random_n1"]["aniso"]["diff_disc"].copy()

In [ ]:
figs = {}
for i_random in tqdm(path_random):
    figs[i_random] = {}
    _fig, _axes = plt.subplots(len(config.file_names), 2, figsize=(14, 20))
    for idx, (i_name, i_content) in enumerate(list(data_pij[i_random].items())):
        pij_true = i_content["pij_true"].copy()
        abilities = data_params[i_random][i_name]["abilities"]
        diffs = data_params[i_random][i_name]["diff_disc"]
        
        _params_plot = [
            {
                "x": _param[0],
                "y": _param[1],
            } for _param in [
                    [pij_true.mean().values, abilities["abilities"].values],
                    [1 - pij_true.mean(axis=1).values, diffs["difficulty"].values],
                ]
        ]
        
        for idx_param, _param in enumerate(_params_plot):
            if idx_param == 1:
                _param = _param|{"c": diffs["discrimination"].values}
                scatter = _axes[idx, idx_param].scatter(**_param)
                cbar = plt.colorbar(scatter, ax=_axes[idx, idx_param])
                cbar.set_label("discrimination")
            else:
                scatter = _axes[idx, idx_param].scatter(**_param)
            _axes[idx, idx_param].grid(True)
            _axes[idx, idx_param].set_title(i_name + " | {:.5}".format(np.corrcoef(_param['x'], _param['y'])[0,1]), loc='center')
            
        _axes[idx, 0].set_ylabel("abilities")
        _axes[idx, 1].set_ylabel("difficulty")
    
    _axes[-1, 0].set_xlabel("average_response")
    _axes[-1, 1].set_xlabel("1 - average_item")

    plt.tight_layout()
    plt.ioff()
    figs[i_random] = {
        "figure": _fig,
        "file_path": (
            path_outputs
            / Path("_".join([i_random,"abi", "diff", "versus"]) + ".png")
        )
    }


## Save

In [ ]:
for contents in( figs.values()):
    _fig, _file_path = contents["figure"], contents["file_path"]
    _fig.savefig(_file_path)

In [ ]:
figs[i_random]["figure"]